In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import gc
import sys
import math

from pandas.io.json import json_normalize
from datetime import datetime

import os
pd.set_option('display.max_columns', 500)

import gc
import os
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
import time
from ast import literal_eval

In [100]:
gc.enable()

# features = ['channelGrouping', 'date', 'fullVisitorId', 'visitId',\
#        'visitNumber', 'visitStartTime', 'device.browser',\
#        'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',\
#        'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',\
#        'geoNetwork.metro', 'geoNetwork.networkDomain', 'geoNetwork.region',\
#        'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',\
#        'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',\
#        'trafficSource.adContent', 'trafficSource.campaign',\
#        'trafficSource.isTrueDirect', 'trafficSource.keyword',\
#        'trafficSource.medium', 'trafficSource.referralPath',\
#        'trafficSource.source']

def load_df(csv_path,nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    ans = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
            converters={column: json.loads for column in JSON_COLUMNS}, 
            dtype={'fullVisitorId': 'str'}, # Important!!
            chunksize=10000,nrows=nrows)
    for df in dfs:
        df.reset_index(drop = True,inplace = True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        # Normalize customDimensions
        df['customDimensions']=df['customDimensions'].apply(literal_eval)
        df['customDimensions']=df['customDimensions'].str[0]
        df['customDimensions']=df['customDimensions'].apply(lambda x: {'index':np.NaN,'value':np.NaN} if pd.isnull(x) else x)

        column_as_df = json_normalize(df['customDimensions'])
        column_as_df.columns = [f"customDimensions_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop('customDimensions', axis=1).merge(column_as_df, right_index=True, left_index=True)
        # Normalize hits
        feat = 'hits'
        df[feat]=df[feat].apply(literal_eval)
        df[feat]=df[feat].str[0]
#         print(df[feat])
        column_as_df = json_normalize(df[feat])
        column_as_df.columns = [f"hits_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop('hits', axis=1).merge(column_as_df, right_index=True, left_index=True)
            
            
        print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
        use_df = df
        del df
        gc.collect()
        ans = pd.concat([ans, use_df], axis=0).reset_index(drop=True)
        #print(ans.shape)
    return ans

In [101]:
%%time
train = load_df('./data/train_v2.csv',nrows=100)

# test = load_df('../input/test_v2.csv')

# print('train date:', min(train['date']), 'to', max(train['date']))
# print('test date:', min(test['date']), 'to', max(test['date']))

Loaded train_v2.csv. Shape: (100, 107)
CPU times: user 232 ms, sys: 0 ns, total: 232 ms
Wall time: 232 ms


In [105]:
train.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,device.deviceCategory,device.flashVersion,device.isMobile,device.language,device.mobileDeviceBranding,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.mobileDeviceModel,device.mobileInputSelector,device.operatingSystem,device.operatingSystemVersion,device.screenColors,device.screenResolution,geoNetwork.city,geoNetwork.cityId,geoNetwork.continent,geoNetwork.country,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.networkLocation,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,totals.visits,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,customDimensions_index,customDimensions_value,hits_appInfo.exitScreenName,hits_appInfo.landingScreenName,hits_appInfo.screenDepth,hits_appInfo.screenName,hits_contentGroup.contentGroup1,hits_contentGroup.contentGroup2,hits_contentGroup.contentGroup3,hits_contentGroup.contentGroup4,hits_contentGroup.contentGroup5,hits_contentGroup.contentGroupUniqueViews1,hits_contentGroup.contentGroupUniqueViews2,hits_contentGroup.previousContentGroup1,hits_contentGroup.previousContentGroup2,hits_contentGroup.previousContentGroup3,hits_contentGroup.previousContentGroup4,hits_contentGroup.previousContentGroup5,hits_customDimensions,hits_customMetrics,hits_customVariables,hits_dataSource,hits_eCommerceAction.action_type,hits_eCommerceAction.step,hits_exceptionInfo.isFatal,hits_experiment,hits_hitNumber,hits_hour,hits_isEntrance,hits_isExit,hits_isInteraction,hits_item.currencyCode,hits_minute,hits_page.hostname,hits_page.pagePath,hits_page.pagePathLevel1,hits_page.pagePathLevel2,hits_page.pagePathLevel3,hits_page.pagePathLevel4,hits_page.pageTitle,hits_page.searchCategory,hits_page.searchKeyword,hits_product,hits_promotion,hits_promotionActionInfo.promoIsView,hits_publisher_infos,hits_referer,hits_social.hasSocialSourceReferral,hits_social.socialInteractionNetworkAction,hits_social.socialNetwork,hits_time,hits_transaction.currencyCode,hits_type
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,desktop,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,Western Europe,1,1,1,1,1,NaN,1,NaN,NaN,not available in demo dataset,NaN,NaN,NaN,NaN,(not set),NaN,water bottle,organic,NaN,google,4,EMEA,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0,shop.googlemerchandisestore.com/google+redesig...,(not set),Bags,(not set),(not set),(not set),NaN,1,(entrance),(entrance),(entrance),(entrance),(entrance),[],[],[],web,0,1,True,[],1,17,True,True,True,USD,0,shop.googlemerchandisestore.com,/google+redesign/bags/water+bottles+and+tumblers,/google+redesign/,/bags/,/water+bottles+and+tumblers,,Water Bottles & Tumblers | Drinkware | Google ...,NaN,NaN,"[{'productSKU': 'GGOEGDHC074099', 'v2ProductNa...",{'index': nan},NaN,[],https://www.

In [102]:
feat = 'hits_promotion'
train[feat] = train[feat].str[0]
# train[feat].apply(literal_eval)
# train[feat] = str(train[feat])
train[feat]=train[feat].apply(lambda x: {'index':np.NaN} if pd.isnull(x) else x)
train[feat]

0                                        {'index': nan}
1     {'promoId': 'Apparel Row 1', 'promoName': 'App...
2     {'promoId': 'Apparel Row 1', 'promoName': 'App...
3     {'promoId': 'Apparel Row 1', 'promoName': 'App...
4     {'promoId': 'Apparel Row 1', 'promoName': 'App...
5     {'promoId': 'Apparel Row 1', 'promoName': 'App...
6     {'promoId': 'Apparel Row 1', 'promoName': 'App...
7     {'promoId': 'Apparel Row 1', 'promoName': 'App...
8     {'promoId': 'Apparel Row 1', 'promoName': 'App...
9                                        {'index': nan}
10                                       {'index': nan}
11                                       {'index': nan}
12                                       {'index': nan}
13                                       {'index': nan}
14                                       {'index': nan}
15                                       {'index': nan}
16                                       {'index': nan}
17                                       {'index

In [103]:
kkk = json_normalize(train[feat])

In [104]:
kkk

,index,promoCreative,promoId,promoName,promoPosition
0,NaN,NaN,NaN,NaN,NaN
1,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
2,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
3,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
4,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
5,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
6,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
7,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
8,NaN,home_main_link_apparel.jpg,Apparel Row 1,Apparel,Row 1
9,NaN,NaN,NaN,NaN,NaN


In [107]:
df = pd.read_csv("./data/train.tmp.4",low_memory=False)

In [108]:
df

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserSize,device_browserVersion,device_deviceCategory,device_flashVersion,device_isMobile,device_language,device_mobileDeviceBranding,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_mobileDeviceModel,device_mobileInputSelector,device_operatingSystem,device_operatingSystemVersion,device_screenColors,device_screenResolution,geoNetwork_city,geoNetwork_cityId,geoNetwork_continent,geoNetwork_country,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_networkLocation,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,totals_visits,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source,customDimensions_index,customDimensions_value,hits_appInfo.exitScreenName,hits_appInfo.landingScreenName,hits_appInfo.screenDepth,hits_appInfo.screenName,hits_contentGroup.contentGroup1,hits_contentGroup.contentGroup2,hits_contentGroup.contentGroup3,hits_contentGroup.contentGroup4,hits_contentGroup.contentGroup5,hits_contentGroup.contentGroupUniqueViews1,hits_contentGroup.contentGroupUniqueViews2,hits_contentGroup.contentGroupUniqueViews3,hits_contentGroup.previousContentGroup1,hits_contentGroup.previousContentGroup2,hits_contentGroup.previousContentGroup3,hits_contentGroup.previousContentGroup4,hits_contentGroup.previousContentGroup5,hits_customDimensions,hits_customMetrics,hits_customVariables,hits_dataSource,hits_eCommerceAction.action_type,hits_eCommerceAction.step,hits_eventInfo.eventAction,hits_eventInfo.eventCategory,hits_eventInfo.eventLabel,hits_exceptionInfo.isFatal,hits_experiment,hits_hitNumber,hits_hour,hits_index,hits_isEntrance,hits_isExit,hits_isInteraction,hits_item.currencyCode,hits_minute,hits_page.hostname,hits_page.pagePath,hits_page.pagePathLevel1,hits_page.pagePathLevel2,hits_page.pagePathLevel3,hits_page.pagePathLevel4,hits_page.pageTitle,hits_page.searchCategory,hits_page.searchKeyword,hits_promotionActionInfo.promoIsView,hits_publisher_infos,hits_referer,hits_social.hasSocialSourceReferral,hits_social.socialInteractionNetworkAction,hits_social.socialNetwork,hits_time,hits_transaction.currencyCode,hits_type,hits_promotion_index,hits_promotion_promoCreative,hits_promotion_promoId,hits_promotion_promoName,hits_promotion_promoPosition,hits_product_customDimensions,hits_product_customMetrics,hits_product_index,hits_product_isClick,hits_product_isImpression,hits_product_localProductPrice,hits_product_productBrand,hits_product_productListName,hits_product_productListPosition,hits_product_productPrice,hits_product_productQuantity,hits_product_productSKU,hits_product_productVariant,hits_product_v2ProductCategory,hits_product_v2ProductName
0,Referral,20161112,6835623374871142057,Not Socially Engaged,1479005348,1,1479005348,Chrome,not available in demo dataset,not available in demo dataset,mobile,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,Android,not available in demo dataset,not available in demo dataset,not available in demo dataset,Bangkok,not available in demo dataset,Asia,Thailand,not available in demo dataset,not available in demo dataset,(not set),unknown.unknown,not available in demo dataset,Bangkok,Southeast Asia,NaN,13,1.0,8.0,NaN,298.0,NaN,NaN,NaN,1,NaN,NaN,not available

In [109]:
df.shape

(10000, 133)